# Assignment: MTA Arrival Clock

In the "MTA Arrival Clock" case study, we have created an alternative design to visualize the arrival time information that looks at the following:

<div><img src="https://user-images.githubusercontent.com/3606672/246465520-5b7cad68-4722-4192-ad69-d0916c8f9cea.svg" width=400/></div>

As noted in the case study, it has the following drawbacks:

* Only trains arriving within the next 10 min are displayed.
* When two or more trains have the same arrival time, the subway line symbols and corresponding annotations will overlap.

## Task:

In this assignment, your task is to come up with a new design that addresses at least one of the two aforementioned drawbacks. You can either use the existing dot plot design or come up with a complete new design. The requirements of the design is the following:

* The dimension of the visualization is fixed (833x200).
* The design should make it easy to extract the arrival time information for each of the upcoming trains.
* The arrival time information should be legible from afar.
* Be mindful about utilizing the available spaces.

Please use this notebook for this assignment.

In [1]:
import altair as alt
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "line": ["F", "F", "E", "M"],
    "arrival_time": [3, 6, 6, 0], 
    "destination": ["Jamaica-179 St", "Kings Hwy", "Jamaica-179 St", "Forest Hills-71 Av"]})
line_color = pd.DataFrame({
    "line": ["E", "F", "M"],
    "color": ["#0039a6", "#ff6319", "#ff6319"]
})

In [ ]:
# Create a vertical offset for overlapping trains
df['y_offset'] = 0
df.loc[(df['arrival_time'] == 6) & (df['line'] == 'E'), 'y_offset'] = -30
df.loc[(df['arrival_time'] == 6) & (df['line'] == 'F'), 'y_offset'] = 30

In [23]:
# Base chart settings
alt.data_transformers.disable_max_rows()

# circles
circles = alt.Chart(df).mark_circle(size=1000).encode(
    x=alt.X('arrival_time:Q', 
            scale=alt.Scale(domain=[0, 10]),
            axis=None),
    y=alt.Y('y_offset:Q', 
            scale=alt.Scale(domain=[-50, 50]),
            axis=None),
    color=alt.Color('line:N',
                   scale=alt.Scale(domain=line_color['line'].tolist(),
                                 range=line_color['color'].tolist())),
    tooltip=['line', 'arrival_time', 'destination']
)

# train lines
text = alt.Chart(df).mark_text(
    align='center',
    baseline='middle',
    fontSize=20,
    fill='white'
).encode(
    x='arrival_time:Q',
    y='y_offset:Q',
    text='line',
    tooltip=['line', 'arrival_time', 'destination']
)

# destination 
destination = alt.Chart(df).mark_text(
    align='center',
    baseline='middle',
    fontSize=12,
    dy=-25
).encode(
    x='arrival_time:Q',
    y='y_offset:Q',
    text='destination',
    color=alt.value('#666666')
)

# arrival time text with "mins" 
arrival_time_text = alt.Chart(df).transform_calculate(
    arrival_time_with_units="datum.arrival_time + ' mins'"
).mark_text(
    align='center',
    baseline='top',
    fontSize=12,  # Increased font size for readability
    dy=20  # Offset below the circles
).encode(
    x='arrival_time:Q',
    y='y_offset:Q',
    text='arrival_time_with_units:N',
    color=alt.value('#333333')
)

# Combine
final_chart = (circles + text + destination + arrival_time_text).properties(
    width=833,
    height=200,
    title='MTA Train Arrivals'
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    ticks=False,
    labels=False
)

final_chart


alt.LayerChart(...)